In [ ]:
import pandas as pd
import logging

In [ ]:
# Logging config
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load the Excel file
file_path = 'СправочникНовый.xlsx'
excel_data = pd.ExcelFile(file_path)

# Load the sheet
sheet_name = 'Справочник ВР'
df = excel_data.parse(sheet_name)

# Define the VR categories and their corresponding row numbers (From the excel, visual)
vr_categories_updated = {
    'Подготовительные работы': 3,
    'Земляные работы': 72,
    'Устройство фундаментов': 77,
    'Устройство сборно-монолитного каркаса': 280,
    'Кладка стен и перегородок из мелкоштучных элементов': 2080,
    'Внутренние инженерные сети': 2616,
    'Устройство внутренних сетей водоснабжения': 2617,
    'Устройство внутренних сетей водоотведения. Хозяйственная канализация': 4714,
    'Устройство внутренних сетей водоотведения. Дождевая (ливневая) канализация': 4982,
    'Устройство внутренних сетей отопления': 5173,
    'Устройство внутренних сетей общеобменной вентиляции': 10121,
    'Устройство внутренних сетей дымоудаления и подпора': 11449,
    'Устройство внутренних сетей кондиционирования': 12794,
    'Устройство перегородок и коробов из листовых материалов': 13239,
}

# Create a new DataFrame for unique entries from column M (Naimenovanie) and the required columns
unique_df = df[['Наименование', 'Тип', 'Материальный ресурс', 'Единица измерения']].drop_duplicates().reset_index(drop=True)
unique_df = unique_df[['Наименование', 'Тип', 'Материальный ресурс', 'Единица измерения']]  # Ensure only relevant columns remain

for vr in vr_categories_updated.keys(): # Add columns for each VR category
    unique_df[vr] = 0 # Initiate it with zeros. later on, we will rewrite

# Populate the VR columns based on the original data
for index, row in df.iterrows():
    for vr, start_row in vr_categories_updated.items():
        end_row = list(vr_categories_updated.values())[list(vr_categories_updated.keys()).index(vr) + 1] if list(vr_categories_updated.keys()).index(vr) + 1 < len(vr_categories_updated) else len(df)
        if start_row <= index < end_row:
            naimenovanie = row['Наименование']
            if pd.notnull(row['Материальный ресурс']):
                logging.info(f"Processing row {index}, VR: {vr}, Start: {start_row}, End: {end_row}, Naimenovanie: {naimenovanie}")
                unique_df.loc[unique_df['Наименование'] == naimenovanie, vr] = 1
                unique_df.loc[unique_df['Наименование'] == naimenovanie, 'Тип'] = 'МР'
            else:
                logging.warning(f"Duplicate entry found for {naimenovanie} in {vr}")
                pass

In [ ]:
unique_df # Intermediate output of the resulting DataFrame

# Shows all fields, independent whether it has empty data or not.

In [ ]:
unique_df = unique_df.dropna(subset=['Материальный ресурс']).reset_index(drop=True) # Drop non MR records

In [ ]:
unique_df # resulting dataframe after dropping non MR entries

In [ ]:
materials_sheet_name = 'Материальные ресурсы' # из Excel Справочник новый (sheet_name)
materials_df = excel_data.parse(materials_sheet_name) # read Excel
materials_df = materials_df.dropna(subset=['Ресурс']).reset_index(drop=True) # Drop non MR records

# Create a set of unique Materialny resurs entries from the unique_df
unique_materials = set(unique_df['Наименование'].dropna().unique())

# Create a set of Materialny resurs entries from the Материальные ресурсы sheet
materials_entries = set(materials_df['Наименование'].dropna().unique())



In [ ]:
print(f' Число материалов в новой таблице справочника: {len(unique_materials)}')
print(f' Число материалов в оригинальной таблице: {len(materials_entries)}')

# Compare the two sets to find missing entries
missing_entries = unique_materials - materials_entries

# Initialize a counter for missing entries
missing_count = len(missing_entries)

# Log missing entries
if missing_count > 0:
    logging.info(f"Number of missing entries: {missing_count}")
    for entry in missing_entries:
        logging.info(f"Missing entry: {entry}")
else:
    logging.info("All entries from unique_df are present in Материальные ресурсы.")

In [ ]:
unique_df # final row

In [ ]:
# Define the VR categories and their corresponding identifiers based on the image provided
vr_identifiers = {
    'Подготовительные работы': '39098adb-e54d-4537-91dc-8b848b9a020e',
    'Земляные работы': '18d66049-7ef5-4dbc-8c76-92e7ed3b6cb9',
    'Устройство фундаментов': 'c23b4467-fa80-47e3-9813-9db288fba930',
    'Устройство сборно-монолитного каркаса': 'b43873bc-a841-4271-98a2-1685bae3dcb4',
    'Кладка стен и перегородок из мелкоштучных элементов': '9f3a9060-b383-4ad5-b49d-1968fa0f810c',
    'Устройство внутренних сетей водоснабжения': 'ec6c1dde-f3da-4ae2-91c5-f76b5dd2593d',
    'Устройство внутренних сетей водоотведения. Хозяйственная канализация': '5503e09f-d6bb-40ee-90c8-1582e897d787',
    'Устройство внутренних сетей водоотведения. Дождевая (ливневая) канализация': '17f5e76e-3eb9-4ef8-a5b9-6bfa9f896242',
    'Устройство внутренних сетей отопления': 'd662dfad-5788-40e0-a2ac-305140d3b443',
    'Устройство внутренних сетей общеобменной вентиляции': '864be648-7273-4ce5-9fe7-d1b70c008206',
    'Устройство внутренних сетей дымоудаления и подпора': 'c6f5f5a0-26de-4cba-af05-2609f930cb80',
    'Устройство внутренних сетей кондиционирования': '36fba20b-26e5-44a5-bc2f-f8108e13ae6f',
    'Устройство перегородок и коробов из листовых материалов': '61f2425b-377c-4bcd-b34a-ac43ee66b57b',
}

# Function to get the UUIDs for each row
def get_vr_identifiers(row):
    identifiers = []
    for vr, uuid in vr_identifiers.items():
        if row[vr] == 1:
            identifiers.append(uuid)
    return ', '.join(identifiers)

# Apply the function to each row in the dataframe
unique_df['VR Identifiers'] = unique_df.apply(get_vr_identifiers, axis=1)
unique_df[['Наименование','Единица измерения','VR Identifiers']]


In [ ]:
output_file_path_updated = 'Новый Список Материалов Е Курылыс.xlsx'
unique_df.to_excel(output_file_path_updated, index=False)
print("Updated file saved to:", output_file_path_updated)

In [ ]:
new_spreadsheet = 'Список материалов Е Курылыс Трансформированный.xlsx'

transformed_mats = pd.ExcelFile(new_spreadsheet).parse('Sheet1')

transformed_mats = transformed_mats[['Наименование', 'Тип', 'Единица измерения', 'VR Identifiers']]

data = {
    "шт": "b73e69d8-9865-4513-9247-34f8176cf06b",
    "м2": "852b8f74-ac8a-4845-a3c4-b553a741c086",
    "м3": "0ec1268f-9b3a-48da-a9c9-2a89460d7b77",
    "м": "7f3868df-eeb2-4392-809f-1c1d70404155",
    "м.п.": "3d17713e-431d-4633-ba91-c36836419355",
    "тн": "8486fddd-b144-4ab8-8b44-0e0a136a4299",
    "комплект": "0ce19e4a-89fb-4377-8a05-e9225ceb75b9",
    "кг": "61d45879-00d8-40f6-9459-d9aa43286daa",
    "отверстие": "8e0e3761-cc7f-4026-a751-000968c808d1"
}

# Function to get the UUIDs for each row
def get_type_identifiers(row):
    identifiers = []
    for vr, uuid in data.items():
        if row['Единица измерения'] == vr:
            identifiers.append(uuid)
    return identifiers

# Apply the function to each row in the dataframe
unique_df['Ed. Izmerenija Identifiers'] = unique_df.apply(get_type_identifiers, axis=1)

In [ ]:
transformed_mats

In [ ]:

armatura = unique_df[unique_df['Наименование'].str.contains("Арматура", na=False, case = True)].reset_index(drop=True)
armatura = armatura[['Наименование', 'Тип', 'Материальный ресурс', 'VR Identifiers']]
armatura

In [ ]:
output_file_path_updated = 'список арматура.xlsx'
armatura.to_excel(output_file_path_updated, index=False)
print("Updated file saved to:", output_file_path_updated)

In [ ]:
# vr_categories_ids = {
#     'Подготовительные работы': 3,
#     'Земляные работы': 62,
#     'Устройство фундаментов': 67,
#     'Устройство сборно-монолитного каркаса': 463,
#     'Кладка стен и перегородок из мелкоштучных элементов': 2993,
#     'Внутренние инженерные сети': 3528,
#     'Устройство внутренних сетей водоснабжения': 3529,
#     'Устройство внутренних сетей водоотведения. Хозяйственная канализация': 5626,
#     'Устройство внутренних сетей водоотведения. Дождевая (ливневая) канализация': 5894,
#     'Устройство внутренних сетей отопления': 6085,
#     'Устройство внутренних сетей общеобменной вентиляции': 11033,
#     'Устройство внутренних сетей дымоудаления и подпора': 12361,
#     'Устройство внутренних сетей кондиционирования': 13706,
#     'Устройство перегородок и коробов из листовых материалов': 14151,
# } 
# Old record numbers

In [ ]:
import pandas as pd
import logging

nomenklatura_name = 'номенклатура.xlsx'

excel = pd.ExcelFile(nomenklatura_name)

nomenklatura_df = excel.parse("TDSheet")

In [ ]:
nomenklatura_df = nomenklatura_df[['Код', 'Наименование', 'Ед. изм.']]

unique_items = set(nomenklatura_df['Наименование'].dropna().unique())

nomenklatura_df

In [ ]:
len(unique_items)

In [ ]:
import pandas as pd
# Logging config
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load the Excel file
file_path = 'Статьи затрат.xlsx'
excel_data = pd.ExcelFile(file_path)

# Load the sheet
sheet_name = 'Лист2'
df = excel_data.parse(sheet_name)

# Define the VR categories and their corresponding row numbers (From the excel, visual)
vr_categories_updated = {
    'Административные расходы': 0,
    'Возврат средств': 20,
    'Зарплата': 23,
    'Инвестиции в проект': 31,
    'Накладные расходы': 35,
    'Налоги': 48,
    'Переток средств': 66,
    'ПИР': 68,
    'СМР': 78,
    'ТМЦ': 108,
    'Финансовые затраты': 140,
    'Коммерческие расходы': 145,
}

expenses = {
    "Финансовые затраты": "93cbdede-7410-414f-aa08-b8444a71540b",
    "ТМЦ": "a2f43068-5748-477c-963e-ff3936d38ecc",
    "СМР": "ec192c19-2e73-4976-884c-95c0870a840e",
    "ПИР": "7945f171-9467-4116-be13-032892c597fe",
    "Переток средств": "e9e6e176-be86-4996-8be8-13961994dbe0",
    "Налоги": "ecdtfce9-7bf3-41de-842a-0c3c787b2ee8",
    "Накладные расходы": "660d7d01-ae83-46d3-b8ea-7977763c9e72",
    "Инвестиции в проект": "Oaeb23e6-fd4b-444e-851d-db59ac250e4d",
    "Зарплата": "77f4bb6-46e6-40b9-8ea2-db402efa8ea8",
    "Возврат средств": "9870d056-1163-46c6-8905-0690ad3b6b45",
    "Административные расходы": "4719ce35-99c8-4386-a512-a268d1f408f2",
    "Коммерческие расходы": "2cfe3d77-8f33-4a77-b430-Flceaba47d9c"
}

# Create a new DataFrame for unique entries from column M (Naimenovanie) and the required columns
unique_df = df[['Виды затрат', 'ТРУ']].drop_duplicates().reset_index(drop=True)

for vr in expenses.keys(): # Add columns for each VR category
    unique_df[vr] = 0 # Initiate it with zeros. later on, we will rewrite

# Populate the VR columns based on the original data
for index, row in df.iterrows():
    for vr, start_row in vr_categories_updated.items():
        end_row = list(vr_categories_updated.values())[list(vr_categories_updated.keys()).index(vr) + 1] if list(vr_categories_updated.keys()).index(vr) + 1 < len(vr_categories_updated) else len(df)
        if start_row <= index < end_row:
            naimenovanie = row['ТРУ']
            if pd.notnull(row['ТРУ']):
                logging.info(f"Processing row {index}, VR: {vr}, Start: {start_row}, End: {end_row}, Naimenovanie: {naimenovanie}")
                unique_df.loc[unique_df['ТРУ'] == naimenovanie, vr] = 1
            else:
                logging.warning(f"Duplicate entry found for {naimenovanie} in {vr}")
                pass

# Function to get the UUIDs for each row
def get_type_identifiers(row):
    identifiers = []
    for vr, uuid in expenses.items():
        if row[vr] == 1:
            identifiers.append(uuid)
    return ', '.join(identifiers)

# Apply the function to each row in the dataframe
unique_df['TRU_ID'] = unique_df.apply(get_type_identifiers, axis=1)

In [ ]:
unique_df = unique_df.dropna(subset=['ТРУ']).reset_index()
output_file_path_updated = 'docv_tru.xlsx'
unique_df.to_excel(output_file_path_updated, index=False)
print("Updated file saved to:", output_file_path_updated)

In [ ]:
unique_df